# Практична робота №1. Отримання навичок роботи в середовищi Python
Варіант 25 \
Виконали студенти групи КА-35 Бернацький Артем, Бубало Анна, Іванкевич Богдан, Шальман Тимофій

### Ініціалізація потрібних змінних та значень

In [7]:
# Множина товарів
I = {"молоко", "вода", "борошно", "яблука"}

# Множина транзакцій
D = [["молоко", "вода", "борошно"],
    ["молоко", "борошно"],
    ["молоко", "вода", "борошно","яблука"],
    ["борошно","вода"],
    ["яблука"]]

# Поріг мінімальної підтримки
supp_min = 0.4

### Реалізація функції $Supp$
$Supp(F) = \frac{|D_F|}{|D|}$

In [8]:
def supp(F, D):
    """
    Функція знаходження підтримки повертає відношення кількості елементів
    з множини товарів до кількості транзакцій.

    Аргументи:
        F - множина товарів, для якої рахуємо підтримку.
        D - множина транзакцій, де кожен елемент D - одна транзакція.
    """

    count = 0
    for row in D:
        if all(i in row for i in F):
            count += 1
    return count / len(D)

### Розрахунок $L_1$
$L_1 = \{ i|i \in{I}, Supp(i) \ge Supp_{min} \}$

In [9]:
# Для кожного елемента множини товарів знаходимо supp.
# Отримуємо множину одноелементних наборів, це необхідно для коректної роботи функції supp.

L1 = [[i] for i in I if supp([i], D) >= supp_min]
L1

[['вода'], ['борошно'], ['яблука'], ['молоко']]

### Реалізація алгоритму знаходження результуючої множини частих наборiв $L_k$
$L_k=\{ F \cup{i} | F \in{L_{k-1}}, i \in{L_1} \setminus F, Supp(F \cup{ \{i\} }) \ge Supp_{min} \}$

In [10]:
def get_frequent_sets(D, L1, supp_min):
    """
    Функція get_frequent_sets знаходить усі часті множини елементів у транзакціях D.

    Аргументи:
        L1 - множина L1, яку ми рахували раніше.
        D - множина транзакцій, де кожен елемент D - одна транзакція.
        supp_min - поріг мінімальної підтримки.

    Алгоритм:
        1. Створюємо початковий список результатів result - у ньому зберігатимемо кортежі
        виду (підтримка, множина) для кожної знайденої множини.
        2. Обчислюємо підтримку для L1 та зберігаємо у result.
        3. Створюємо змінну для попередньої множини L_prev як копію L1.
        4. Запускаємо цикл:
            - Створюємо порожній список Lk для збереження нової множини.
            - Для кожного елементу F з L_prev та для кожного i з L1:
                1) Оскільки множина L1 складається з одноелементних наборів,
                щоб отримати елемент ми беремо нульовий елемент списку item = i[0].
                2) Якщо item ще не входить у F, формуємо нову множину F_new додаючи item до F
                3) Сортуємо результат, щоб уникнути однакових комбінацій товарів в множинах.
                4) Якщо множини F_new ще немає в Lk:
                    - Рахуємо підтримку supp_f = supp(F_new, D).
                    - Якщо supp_f ≥ supp_min, додаємо F_new у Lk і записуємо (supp_f, F_new) у result.
            - Якщо на цьому кроці не вдалося знайти жодного нового частого набору (Lk порожній), то припиняємо цикл.
            - Інакше оновлюємо L_prev = Lk і повторюємо процес.
        5. Повертаємо result - усі знайдені часті набори з їх підтримками.
    """

    result = [(supp(F, D), F) for F in L1]
    L_prev = L1.copy()

    while True:
        Lk = []
        for F in L_prev:
            for i in L1:
                item = i[0]
                if item not in F:
                    F_new = sorted(F + [item])  #  F ∪ {i}, де i ∈ L1 \ F
                    if F_new not in Lk:
                        supp_f = supp(F_new, D)
                        if supp_f >= supp_min:
                            Lk.append(F_new)
                            result.append((supp_f, F_new))
        if not Lk:
            break

        L_prev = Lk

    return result

### Результат виконання алгоритму на нашому датасеті

In [11]:
frequent_sets = get_frequent_sets(D, L1, supp_min)
print(" Supp\tSet")
frequent_sets

 Supp	Set


[(0.6, ['вода']),
 (0.8, ['борошно']),
 (0.4, ['яблука']),
 (0.6, ['молоко']),
 (0.6, ['борошно', 'вода']),
 (0.4, ['вода', 'молоко']),
 (0.6, ['борошно', 'молоко']),
 (0.4, ['борошно', 'вода', 'молоко'])]